In [12]:
# ============================================
# NYC 24/7 Cameras — OLS (month & hour FE) + Ridge/Lasso diagnostics
# Data window: ±6 months around Aug 1, 2022
# Outputs: LaTeX tables + PDF figures for LaTeX
# ============================================

import warnings
warnings.simplefilter('ignore')

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from scipy import sparse

# ------------------ Config ------------------
data_path = "/Users/eamon/Desktop/University/UofT 2025-26/Fall/Applied Machine Learning/Research Project/Original Data/Motor_Vehicle_Collisions_-_Crashes_20250917.csv"
report_dir = "/Users/eamon/Desktop/University/UofT 2025-26/Fall/Applied Machine Learning/Week 3/Report"
os.makedirs(report_dir, exist_ok=True)

policy_date = pd.Timestamp("2022-08-01")
start_date  = policy_date - pd.DateOffset(months=6)
end_date    = policy_date + pd.DateOffset(months=6)

valid_boros = {"BRONX", "BROOKLYN", "MANHATTAN", "QUEENS", "STATEN ISLAND"}

# -------------- Load & Prepare --------------
nyc_mvc = pd.read_csv(data_path)

# Time fields
nyc_mvc["date"] = pd.to_datetime(nyc_mvc["CRASH DATE"], errors="coerce")
nyc_mvc["hour"] = pd.to_datetime(nyc_mvc["CRASH TIME"], format="%H:%M", errors="coerce").dt.hour
nyc_mvc = nyc_mvc.dropna(subset=["date", "hour"]).copy()
nyc_mvc["hour"] = nyc_mvc["hour"].astype(int)

# Borough cleaning + filter to real NYC boroughs
nyc_mvc["BOROUGH"] = nyc_mvc["BOROUGH"].astype(str).str.upper().str.strip()
nyc_mvc = nyc_mvc[nyc_mvc["BOROUGH"].isin(valid_boros)].copy()
nyc_mvc["borough"] = nyc_mvc["BOROUGH"]

# Restrict to ±6 months around policy date
nyc_mvc = nyc_mvc[(nyc_mvc["date"] >= start_date) & (nyc_mvc["date"] <= end_date)].copy()

# DiD indicators
nyc_mvc["after"] = (nyc_mvc["date"] >= policy_date).astype(int)
nyc_mvc["night"] = ((nyc_mvc["hour"] >= 22) | (nyc_mvc["hour"] < 6)).astype(int)
nyc_mvc["after_night"] = nyc_mvc["after"] * nyc_mvc["night"]

# Month fixed effects label
nyc_mvc["month"] = nyc_mvc["date"].dt.to_period("M").astype(str)

# Vehicle-type control (collapsed classes)
def map_vehicle(s):
    if pd.isna(s):
        return "OTHER"
    s = str(s).upper().strip()
    if "SEDAN" in s or "PASSENGER" in s or "CONVERTIBLE" in s or "3-DOOR" in s:
        return "CAR"
    if "SPORT UTILITY" in s or "STATION WAGON" in s:
        return "SUV_WAGON"
    if "TAXI" in s or "LIVERY" in s:
        return "TAXI"
    if "PICK" in s or "VAN" in s or "CARRY ALL" in s or "REFRIGERATED VAN" in s:
        return "LIGHT_TRUCK_VAN"
    if ("TRUCK" in s and "PICK" not in s) or "TRACTOR" in s or "DUMP" in s or \
       "FLAT" in s or "WRECKER" in s or "GARBAGE" in s or "CONCRETE" in s or \
       "ARMORED" in s or "BEVERAGE" in s or "CHASSIS" in s or "TANKER" in s or \
       "LARGE COM" in s or "SMALL COM" in s:
        return "TRUCK"
    if "BUS" in s:
        return "BUS"
    if "BIKE" in s or "BICYCLE" in s:
        return "BICYCLE"
    if "MOTORCYCLE" in s or "MOTORBIKE" in s or "SCOOTER" in s or "MOPED" in s:
        return "MOTORCYCLE"
    if "E-BIKE" in s or "EBIKE" in s or "E-SCOOT" in s:
        return "MICROMOBILITY"
    if "AMBULANCE" in s or "FIRE" in s:
        return "EMERGENCY"
    return "OTHER"

nyc_mvc["veh_cat"] = (
    nyc_mvc["VEHICLE TYPE CODE 1"]
    .combine_first(nyc_mvc["VEHICLE TYPE CODE 2"])
    .map(map_vehicle)
)

# Outcome
nyc_mvc["injured"] = pd.to_numeric(nyc_mvc["NUMBER OF PERSONS INJURED"], errors="coerce")
nyc_mvc = nyc_mvc.dropna(subset=["injured"]).copy()

# -------------- OLS (exact spec) --------------
# injured ~ after_night + C(veh_cat) + C(borough) + C(month) + C(hour)
formula = "injured ~ after_night + C(veh_cat) + C(borough) + C(month) + C(hour)"
ols_model = smf.ols(formula, data=nyc_mvc).fit(
    cov_type="cluster",
    cov_kwds={"groups": nyc_mvc["borough"]}
)

# Full OLS table → LaTeX + TXT
try:
    full_tex = ols_model.summary().as_latex()
except Exception:
    full_tex = ols_model.summary().as_text()

with open(os.path.join(report_dir, "ols_full.tex"), "w") as f:
    f.write(full_tex)
with open(os.path.join(report_dir, "ols_full.txt"), "w") as f:
    f.write(ols_model.summary().as_text())

# Small OLS table (selected coefficients) → LaTeX + CSV
sel = [
    "Intercept",
    "after_night",
    "C(veh_cat)[T.EMERGENCY]",
    "C(veh_cat)[T.BUS]",
    "C(veh_cat)[T.CAR]",
    "C(veh_cat)[T.TRUCK]",
]
pretty = {
    "Intercept": "Intercept",
    "after_night": "After × Night",
    "C(veh_cat)[T.EMERGENCY]": "Vehicle: Emergency",
    "C(veh_cat)[T.BUS]":       "Vehicle: Bus",
    "C(veh_cat)[T.CAR]":       "Vehicle: Car",
    "C(veh_cat)[T.TRUCK]":     "Vehicle: Truck",
}
coefs = ols_model.params.reindex(sel)
ses   = ols_model.bse.reindex(sel)
tvals = coefs / ses
pvals = ols_model.pvalues.reindex(sel)

small_tbl = pd.DataFrame({
    "Variable":    [pretty.get(i, i) for i in sel],
    "Coefficient": coefs.values,
    "Std. Error":  ses.values,
    "t-Statistic": tvals.values,
    "p-Value":     pvals.values,
})
small_tbl.to_csv(os.path.join(report_dir, "ols_selected_minimal.csv"), index=False)

fmt_tbl = small_tbl.copy()
for c in ["Coefficient", "Std. Error", "t-Statistic", "p-Value"]:
    fmt_tbl[c] = fmt_tbl[c].map(lambda x: f"{x:.4f}")
with open(os.path.join(report_dir, "ols_selected_minimal.tex"), "w") as f:
    f.write(fmt_tbl.to_latex(index=False, escape=False))

# -------------- Design matrix for Ridge/Lasso --------------
# Keep: after_night + FE (veh_cat, borough, month, hour)
# Drop: standalone after, night
y = nyc_mvc["injured"].values
X_df = pd.get_dummies(
    nyc_mvc[["veh_cat", "borough", "month", "hour"]],
    drop_first=True
)
X_df = pd.concat([pd.Series(nyc_mvc["after_night"], name="after_night"), X_df], axis=1)
feature_names = X_df.columns.tolist()

scaler = StandardScaler(with_mean=False)
X_scaled = scaler.fit_transform(X_df)

# Lasso requires dense; Ridge can use sparse. Make a dense copy for both to be consistent.
if sparse.issparse(X_scaled):
    X_dense = X_scaled.toarray()
else:
    X_dense = np.asarray(X_scaled)

# -------------- Ridge: coefficient path (key covariates) --------------
ridge_alphas = np.logspace(-3, 3, 100)
ridge_coefs = []
for a in ridge_alphas:
    ridge = Ridge(alpha=a, fit_intercept=True)
    ridge.fit(X_dense, y)
    ridge_coefs.append(ridge.coef_)
ridge_coefs = np.array(ridge_coefs)

key_features = ["after_night", "veh_cat_BUS", "veh_cat_CAR", "veh_cat_TRUCK", "veh_cat_EMERGENCY"]
key_idx = [i for i, f in enumerate(feature_names) if f in key_features]

plt.figure(figsize=(9, 6))
for i in key_idx:
    plt.plot(ridge_alphas, ridge_coefs[:, i], lw=2, label=feature_names[i])
plt.xscale("log")
plt.xlabel("Lambda (α)")
plt.ylabel("Coefficient value")
plt.title("Ridge Coefficient Paths (key covariates)")
plt.axhline(0, color="black", lw=0.6)
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(report_dir, "ridge_coeff_paths.pdf"), bbox_inches="tight")
plt.close()

# -------------- Ridge: CV curve (MSE with SE bars) --------------
alphas_cv = np.logspace(-3, 3, 50)
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_means, cv_stds = [], []
for a in alphas_cv:
    ridge = Ridge(alpha=a, fit_intercept=True)
    scores = cross_val_score(ridge, X_dense, y, cv=kf, scoring="neg_mean_squared_error")
    mse = -scores
    cv_means.append(mse.mean())
    cv_stds.append(mse.std())
cv_means = np.array(cv_means)
cv_stds = np.array(cv_stds)
best_alpha_ridge = alphas_cv[np.argmin(cv_means)]

plt.figure(figsize=(9, 6))
plt.errorbar(alphas_cv, cv_means, yerr=cv_stds, fmt="-o", capsize=4)
plt.xscale("log")
plt.xlabel("Lambda (α)")
plt.ylabel("Cross-Validation MSE")
plt.title("Ridge CV — MSE with Standard Errors")
plt.grid(True, which="both", ls="--", lw=0.5)
plt.axvline(best_alpha_ridge, color="red", linestyle="--", label=f"Best λ = {best_alpha_ridge:.3f}")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(report_dir, "ridge_cv_mse.pdf"), bbox_inches="tight")
plt.close()

# -------------- Lasso: coefficient path (key covariates) --------------
lasso_alphas = np.logspace(-3, 1, 60)
lasso_coefs = []
for a in lasso_alphas:
    lasso = Lasso(alpha=a, fit_intercept=True, max_iter=5000)
    lasso.fit(X_dense, y)
    lasso_coefs.append(lasso.coef_)
lasso_coefs = np.array(lasso_coefs)

plt.figure(figsize=(9, 6))
for i in key_idx:
    plt.plot(lasso_alphas, lasso_coefs[:, i], lw=2, label=feature_names[i])
plt.xscale("log")
plt.xlabel("Lambda (α)")
plt.ylabel("Coefficient value")
plt.title("Lasso Coefficient Paths (key covariates)")
plt.axhline(0, color="black", lw=0.6)
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(report_dir, "lasso_coeff_paths.pdf"), bbox_inches="tight")
plt.close()

# -------------- Lasso: CV curve (MSE with SE bars) --------------
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_means_l, cv_stds_l = [], []
for a in lasso_alphas:
    mses = []
    for tr, te in kf.split(X_dense):
        lasso = Lasso(alpha=a, fit_intercept=True, max_iter=5000)
        lasso.fit(X_dense[tr], y[tr])
        y_hat = lasso.predict(X_dense[te])
        mses.append(mean_squared_error(y[te], y_hat))
    cv_means_l.append(np.mean(mses))
    cv_stds_l.append(np.std(mses))
cv_means_l = np.array(cv_means_l)
cv_stds_l = np.array(cv_stds_l)
best_alpha_lasso = lasso_alphas[np.argmin(cv_means_l)]

plt.figure(figsize=(9, 6))
plt.errorbar(lasso_alphas, cv_means_l, yerr=cv_stds_l, fmt="-o", capsize=4)
plt.xscale("log")
plt.xlabel("Lambda (α)")
plt.ylabel("Cross-Validation MSE")
plt.title("Lasso CV — MSE with Standard Errors")
plt.grid(True, which="both", ls="--", lw=0.5)
plt.axvline(best_alpha_lasso, color="red", linestyle="--", label=f"Best λ = {best_alpha_lasso:.3f}")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(report_dir, "lasso_cv_mse.pdf"), bbox_inches="tight")
plt.close()

# -------------- Save brief run log --------------
with open(os.path.join(report_dir, "run_summary.txt"), "w") as f:
    f.write("OLS formula: injured ~ after_night + C(veh_cat) + C(borough) + C(month) + C(hour)\n")
    f.write(f"Ridge best alpha: {best_alpha_ridge:.6f}\n")
    f.write(f"Lasso best alpha: {best_alpha_lasso:.6f}\n")
    f.write(f"Rows used: {len(nyc_mvc):,}\n")






